In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import math

# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Apply label encoding to the 'state' column
label_encoder = LabelEncoder()
df['state'] = label_encoder.fit_transform(df['state'])

# 'data' contains columns 'sex', 'age', 'state', 'risk_profile', 'invest_goals', 'temp_goals', 'fin_exp', 'fin_avail'
data = df[['sex', 'age', 'state', 'risk_profile', 'invest_goals', 'temp_goals', 'fin_exp', 'fin_avail']].values

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# 'targets' contains columns 'a1' through 'a20'
targets = df[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20']].values

# Split the data into training and testing sets
split_ratio = 0.7  # 70% for training, 30% for testing
split_index = int(len(data) * split_ratio)

X_train, X_test = data[:split_index], data[split_index:]
y_train, y_test = targets[:split_index], targets[split_index:]

# Create a decision tree regressor
model = DecisionTreeRegressor()

# Define the hyperparameter grid
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 10, 20]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameter values
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best hyperparameter values to create the final model
final_model = DecisionTreeRegressor(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = final_model.predict(X_test)

Best Hyperparameters: {'max_depth': 3, 'min_samples_leaf': 20, 'min_samples_split': 5}


In [2]:
sums = []
for row in y_test:
    sums.append(sum(row))
#print('y_test =', sums)
print("y_test average =", sum(sums)/len(sums))

y_test average = 1.0000227272727267


In [3]:
sums = []
for row in y_pred:
    sums.append(sum(row))
#print('y_pred =', sums)
print("y_pred average =", sum(sums)/len(sums))

y_pred average = 1.0000421573949945


In [4]:
# Remove values < 0.01 and Scale predictions to get 1 (100%) as a sum of each row
num_rows = len(y_pred)
num_cols = len(y_pred[0])

for row in range(num_rows):
    for col in range(num_cols):
        if y_pred[row][col] < 0.01:
            y_pred[row][col] = 0

for row in range(num_rows):
    row_sum = sum(y_pred[row])
    for col in range(num_cols):
        y_pred[row][col] = y_pred[row][col] / row_sum

In [5]:
sums = []
for row in y_pred:
    sums.append(sum(row))
#print('y_pred =', sums)
print("y_pred average =", sum(sums)/len(sums))

y_pred average = 1.0


In [6]:
data_for_csv = df[['sex', 'age', 'state', 'risk_profile', 'invest_goals', 'temp_goals', 'fin_exp', 'fin_avail']].values
data_train, data_test = data_for_csv[:split_index], data_for_csv[split_index:]
array1 = data_test
array2 = np.round(np.array(y_pred), 3)

result_array = np.concatenate((array1, array2), axis=1)

np.set_printoptions(suppress=True)
print(result_array)

np.savetxt('HP_RT_output.csv', result_array, delimiter=",", fmt='%.3f')

[[ 1.    41.    33.    ...  0.     0.     0.   ]
 [ 1.    69.    14.    ...  0.     0.     0.   ]
 [ 1.    46.    28.    ...  0.     0.     0.   ]
 ...
 [ 1.    64.    24.    ...  0.     0.     0.   ]
 [ 1.    57.    11.    ...  0.     0.     0.   ]
 [ 0.    51.    14.    ...  0.012  0.     0.   ]]
